## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.




### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
# импортировали библиотеку pandas

from IPython.display import display
# импортировалиа метод display из библиотеки IPython.display  

import warnings
warnings.filterwarnings('ignore')
# отключили предупреждения

In [2]:
data = pd.read_csv('/datasets/data.csv')
# прочитали файл, сохранили в data


In [3]:
data.info() 
# data.tail(50)
# data.head(50)
# просмотрели инфо, вывели 50 строк (и первые, и последние)
data['total_income_raw'] = data['total_income']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
def hesil(d, n):
    return pd.concat([d.head(n), d.sample(n, random_state=88), d.tail(n)], axis=0)
hesil(data, 7)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_raw
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,253875.639453
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,112080.014102
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,145885.952297
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,267628.550329
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,158616.077870
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,255763.565419
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,240525.971920
5603,1,-891.697143,32,среднее,1,женат / замужем,0,F,сотрудник,0,84413.187240,получение дополнительного образования,84413.187240
2547,0,-307.990303,26,высшее,0,Не женат / не замужем,4,F,компаньон,0,101327.374297,строительство жилой недвижимости,101327.374297
9778,0,-332.173988,45,среднее,1,женат / замужем,0,F,сотрудник,0,185984.564168,покупка своего жилья,185984.564168


### Вывод

Тип данных ок; доход - float, можно сделать целым. <br>
Стаж days_employed может быть отрицательным - ошибка ввода? В задании нет вопроса про связи со стажем - не будем учитывать. <br>
Отрицательные значения сделать положительными - может, с помощью модуля. <br>
В столбцах типа object встречается разное написание (регистры, синонимы). <br>
Столбец debt: логическая переменная, 0 - не имел долг, 1 - имел. Увидели опечатку - 20 детей.  <br>
NaN встречается в стаже days_employed и доходе total_income - скорее всего, во всем дф так и скорее всего строки совпадают. <br>
Может быть ввести purpose_id, для этого привести к единому написанию? <br>
Если у нас есть education_id и family_status_id, то education и family_status можно убрать?   <br>


In [5]:
def hesil(d, n):
    return pd.concat([d.head(n), d.sample(n, random_state=88), d.tail(n)], axis=0)

In [6]:
hesil(data, 3)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_raw
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,253875.639453
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,112080.014102
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,145885.952297
5603,1,-891.697143,32,среднее,1,женат / замужем,0,F,сотрудник,0,84413.187240,получение дополнительного образования,84413.187240
2547,0,-307.990303,26,высшее,0,Не женат / не замужем,4,F,компаньон,0,101327.374297,строительство жилой недвижимости,101327.374297
9778,0,-332.173988,45,среднее,1,женат / замужем,0,F,сотрудник,0,185984.564168,покупка своего жилья,185984.564168
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость,89672.561153
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля,244093.050500
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля,82047.418899


<hr style="border: 2px solid orange;" />

### Шаг 2. Предобработка данных

### Обработка пропусков

In [7]:
display(data.isnull().sum())
# находим суммарное количество пропусков: с пропусками только 2 столбца с количественными значениями.
# рассмотрим  доход, детей, возраст, семейное положение 
 

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
total_income_raw    2174
dtype: int64

In [8]:
#  1 доход 
#data['total_income'] = data['total_income'].fillna(0)
data.loc[data['total_income'].isna(), 'total_income'] = data['total_income'].median()
display(data.isnull().sum())  # проверка



children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
total_income_raw    2174
dtype: int64

In [9]:
# 2 дети 
display(data['children'].value_counts())   # нашли значения 20 (76шт) и -1 (47 шт), считаем их за опечатку
data['children'] = data['children'].apply(abs)    # -1 сделаем методом по  модулю 1  
data.loc[data['children'] == 20, 'children'] = 2    # 20 превратили в 2
display(data['children'].value_counts()) # проверка



 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [10]:
# 3 возраст
# print(data['dob_years'].value_counts())   # посмотрим  уникальные значения и количество их упоминаний в каждом столбце
dob_years_avg = (data['dob_years'].mean()).astype('int')  # 100 значений - 0 лет. Заменим на средний возраст, меняем тип 
display('Средний возраст:', dob_years_avg)   # проверка
data.loc[data['dob_years'] == 0, 'dob_years'] = dob_years_avg
display(data['dob_years'].unique())  # проверка


'Средний возраст:'

43

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 59, 29, 60, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75])

In [11]:
# 4 семейное положение
display(data['family_status'].unique())   # проверка, 5 значений ок

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [12]:
data.groupby('income_type').agg({'total_income': 'mean', 'total_income_raw': 'mean'})

,total_income,total_income_raw
income_type,,
безработный,131339.751676,131339.751676
в декрете,53829.130729,53829.130729
госслужащий,168290.760408,170898.309923
компаньон,196683.153073,202417.461462
пенсионер,137972.581061,137127.465690
предприниматель,322090.541240,499163.144947
сотрудник,159754.181986,161380.260488
студент,98201.625314,98201.625314


<hr style="border: 2px solid orange;" />

### Вывод

Совпадает количество NaN days_employed = total_income = 2174. <br> Пропуски не случайны и совпадают, ведь если нет работы - нет  дохода. Доля пропусков - около 10%. <br> Может быть, это люди с неофициальным источником дохода или в декрете, студенты. NaN нужно заменить в последующих шагах. <br>
Потом нужно заменить income_type - безработный? 

### Замена типа данных

In [13]:
data['total_income'] = data['total_income'].astype(int)    # сделаем целым числом доход
data['children'] = data['children'].apply(abs)    #  уберем отрицательные значения, по модулю


### Вывод

Заменили у столбца доходов тип на int, отбросив знаки после запятой, для того чтобы облегчить восприятие.
Количество детей не может быть отрицательным числом, сделали значение положительным.

### Обработка дубликатов

In [14]:
data['education'] = data['education'].str.lower()   # education перевели в нижний регистр
display(data.duplicated().sum())
data.drop_duplicates(inplace = True) 
display(data.duplicated().sum())    # проверка

71

0

### Вывод

Во всем массиве методом drop_duplicates удалили 71 дубликат . Это очень малая доля от всего дф. 
Можно предположить, что они появились при слиянии нескольких таблиц, при невнимательном копировании.

In [15]:
data['education'].nunique() == data['education_id'].nunique()

True

<hr style="border: 2px solid red;" />

### Категоризация данных

In [16]:
from pymystem3 import Mystem   # импорт библиотеки pymystem3
m = Mystem() 
from collections import Counter  # импорт счетчика


In [17]:
purposes_unique = ' '.join(data['purpose'].unique())   
# список уникальныe значений в столбце цели. итоговый результат склеили вызовом метода join().
lemmas = m.lemmatize(purposes_unique)
#print(lemmas) 
display(Counter(lemmas))   # подсчёт числа уникальных значений 

Counter({'покупка': 10,
         ' ': 96,
         'жилье': 7,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 2,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         '\n': 1})

In [18]:
data['purpose_lemm'] = data['purpose'].apply(m.lemmatize)   # новый столбец с лемматизированными целями - "словарь"

# пишем функцию, которая обрабатывает словарь и выделяет категории, т.е. сопоставляем заявление с определенными причинами
def purpose_cat(lemma): 
    if 'жилье' in lemma or 'недвижимость' in lemma:
        return 'недвижимость'
    if 'автомобиль' in lemma:
        return 'автомобиль'
    if 'образование' in lemma:
        return 'образование'
    if 'свадьба' in lemma:
        return 'свадьба'
    if 'ремонт' in lemma:
        return 'ремонт'       
    return 'другое'
 
data['purpose_new'] = data['purpose_lemm'].apply(purpose_cat)
display(data.head(10))  # проверка


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_raw,purpose_lemm,purpose_new
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,253875.639453,"[покупка, , жилье, \n]",недвижимость
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,112080.014102,"[приобретение, , автомобиль, \n]",автомобиль
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,145885.952297,"[покупка, , жилье, \n]",недвижимость
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,267628.550329,"[дополнительный, , образование, \n]",образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,158616.077870,"[сыграть, , свадьба, \n]",свадьба
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,255763.565419,"[покупка, , жилье, \n]",недвижимость
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,240525.971920,"[операция, , с, , жилье, \n]",недвижимость
7,0,-152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,135823.934197,"[образование, \n]",образование
8,2,-6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,95856.832424,"[на, , проведение, , свадьба, \n]",свадьба
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,144425.938277,"[покупка, , жилье, , для, , семья, \n]",недвижимость


In [19]:
display(data['purpose_new'].unique())  # проверка: всего четыре категории


array(['недвижимость', 'автомобиль', 'образование', 'свадьба'],
      dtype=object)

In [20]:
display(data['purpose_new'].value_counts())  # рейтинг причин: более половины связаны с покупкой недвижимости
display(data['purpose_new'].value_counts(normalize=True)*100)   # в процентах

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_new, dtype: int64

недвижимость    50.391535
автомобиль      20.070849
образование     18.705137
свадьба         10.832479
Name: purpose_new, dtype: float64

### Вывод

В итоге лемматизации нашли 4 основные цели взятия кредитов и посчитали их количество.
Около половины связаны с покупкой недвижимости, примерно по двадцать процентов - у автомобилей и у образования, оставшиеся десять процентов относятся к свадьбам.

### Шаг 3. Ответьте на вопросы

### - Есть ли зависимость между наличием детей и возвратом кредита в срок?


In [21]:
data_new = data[['children','family_status', 'debt','total_income','purpose_new']]   # создаем новую рабочую таблицу
display(data_new.head(10)) 

,children,family_status,debt,total_income,purpose_new
0,1,женат / замужем,0,253875,недвижимость
1,1,женат / замужем,0,112080,автомобиль
2,0,женат / замужем,0,145885,недвижимость
3,3,женат / замужем,0,267628,образование
4,0,гражданский брак,0,158616,свадьба
5,0,гражданский брак,0,255763,недвижимость
6,0,женат / замужем,0,240525,недвижимость
7,0,женат / замужем,0,135823,образование
8,2,гражданский брак,0,95856,свадьба
9,0,женат / замужем,0,144425,недвижимость


In [22]:
display(data['debt'].unique())   # проверка

array([0, 1])

In [23]:
display(data_new['children'].value_counts())  # посчитали количество значений, от 1 до 5 нужно объединить в одну категорию

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

In [24]:
data_new['total_income'] = data_new['total_income'].astype(int)
#print(data['debt'].value_counts()) 


In [25]:
# определяем две категории по наличию детей, количество не берем во внимание:

def children_group(children):
    if children < 1:
                return 'бездетные'
    return 'есть дети'

display(children_group(5))    # проверка определения категории
display(children_group(0))    # проверка

 

'есть дети'

'бездетные'

In [26]:
data_new['children_group'] = data_new['children'].apply(children_group)   
# добавляем столбец, где хранятся результаты применения функции
display(data_new.head(10))    # проверка


,children,family_status,debt,total_income,purpose_new,children_group
0,1,женат / замужем,0,253875,недвижимость,есть дети
1,1,женат / замужем,0,112080,автомобиль,есть дети
2,0,женат / замужем,0,145885,недвижимость,бездетные
3,3,женат / замужем,0,267628,образование,есть дети
4,0,гражданский брак,0,158616,свадьба,бездетные
5,0,гражданский брак,0,255763,недвижимость,бездетные
6,0,женат / замужем,0,240525,недвижимость,бездетные
7,0,женат / замужем,0,135823,образование,бездетные
8,2,гражданский брак,0,95856,свадьба,есть дети
9,0,женат / замужем,0,144425,недвижимость,бездетные


In [27]:
display(data_new.groupby('children_group').agg({'debt': 'mean'}))    # нашли процент должников у бездетных и с детьми

,debt
children_group,
бездетные,0.075438
есть дети,0.092082


In [28]:
data_children = data_new.pivot_table(index='children_group', columns='debt', values='total_income', aggfunc='count').fillna(0)
display(data_children.head(10))
# сделали сводную таблицу


debt,0,1
children_group,,
бездетные,13028,1063
есть дети,6685,678


In [29]:
# проверка, нашли долю должников у бездетных и с детьми
data_children[''] = data_children[1] / (data_children[0] + data_children[1])   
display(data_children.head(10)) 

debt,0,1,
children_group,,,
бездетные,13028,1063,0.075438
есть дети,6685,678,0.092082


In [30]:
# определяем три категории по наличию детей

def children_group(children):
    if children < 1:
                return 'бездетные'
    if children <= 2:
                return '1-2 ребенка'    
    return 'многодетные'

data_new['children_group'] = data_new['children'].apply(children_group)    
display(data_new.sample(10))     

,children,family_status,debt,total_income,purpose_new,children_group
21214,0,гражданский брак,0,117844,свадьба,бездетные
8185,0,женат / замужем,0,84347,образование,бездетные
12564,0,женат / замужем,0,145017,недвижимость,бездетные
8273,0,женат / замужем,0,26533,недвижимость,бездетные
17764,2,гражданский брак,0,346909,свадьба,1-2 ребенка
8694,0,женат / замужем,0,83635,образование,бездетные
12272,0,женат / замужем,0,129282,недвижимость,бездетные
3778,0,гражданский брак,0,68515,свадьба,бездетные
18832,2,гражданский брак,0,75700,недвижимость,1-2 ребенка
14033,1,женат / замужем,0,151518,образование,1-2 ребенка


In [31]:
# нашли процент должников  
display(data_new.groupby('children_group').agg({'debt': 'mean'}))    

,debt
children_group,
1-2 ребенка,0.092654
бездетные,0.075438
многодетные,0.081579


### Вывод

Доля должников в группах: у бездетных - 0.075, есть дети - 0.092. <br>
Клиенты с детьми чаще имеют долг - 9%.  <br>
Остальные вопросы сделаю по такому же шаблону. 

### - Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [32]:
data_new = data[['children','family_status', 'debt','total_income','purpose_new']]   # создаем новую рабочую таблицу
display(data.groupby('family_status')['family_status_id'].value_counts())  # 5 групп семейного положения
 

family_status          family_status_id
Не женат / не замужем  4                    2810
в разводе              3                    1195
вдовец / вдова         2                     959
гражданский брак       1                    4151
женат / замужем        0                   12339
Name: family_status_id, dtype: int64

In [33]:
data_family = data_new.pivot_table(index='family_status', columns='debt', values='total_income', aggfunc='count').fillna(0)
display(data_family.head())

debt,0,1
family_status,,
Не женат / не замужем,2536,274
в разводе,1110,85
вдовец / вдова,896,63
гражданский брак,3763,388
женат / замужем,11408,931


In [34]:
data_family[''] = data_family[1] / (data_family[0] + data_family[1])   # нашли долю должников у 4х категорий доходов
display(data_family.head()) 

debt,0,1,
family_status,,,
Не женат / не замужем,2536,274,0.097509
в разводе,1110,85,0.071130
вдовец / вдова,896,63,0.065693
гражданский брак,3763,388,0.093471
женат / замужем,11408,931,0.075452


In [35]:
display(data_new.groupby('family_status').agg({'debt': 'mean'}))    # проверка 

,debt
family_status,
Не женат / не замужем,0.097509
в разводе,0.071130
вдовец / вдова,0.065693
гражданский брак,0.093471
женат / замужем,0.075452


### Вывод

У вдовых заёмщиков меньше долгов, чем у людей браке или разводе. Это очень интересно, ведь на первый взгляд кажется обратное.<br> Люди в браке более дисциплированны. А не состоящие в официальном браке допускают просрочки. 

## - Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [36]:
data_new = data[['children','family_status', 'debt','total_income','purpose_new']]   # создаем новую рабочую таблицу
data_new['total_income'] = data_new['total_income'].astype(int)
display(data_new.tail()) 

,children,family_status,debt,total_income,purpose_new
21520,1,гражданский брак,0,224791,недвижимость
21521,0,женат / замужем,0,155999,автомобиль
21522,1,гражданский брак,1,89672,недвижимость
21523,3,женат / замужем,1,244093,автомобиль
21524,2,женат / замужем,0,82047,автомобиль


In [37]:
# выделяем категории

def income_group(total_income):
    
    if total_income <= 100000:
            return 'низкий'
    if total_income <= 200000:
            return 'средний'
    if total_income <= 300000:
            return 'высокий'
    return 'сверхвысокий' 


# добавляем столбец, где хранятся результаты применения функции
data_new['income_group'] = data_new['total_income'].apply(income_group) 
display(data_new.tail())

,children,family_status,debt,total_income,purpose_new,income_group
21520,1,гражданский брак,0,224791,недвижимость,высокий
21521,0,женат / замужем,0,155999,автомобиль,средний
21522,1,гражданский брак,1,89672,недвижимость,низкий
21523,3,женат / замужем,1,244093,автомобиль,высокий
21524,2,женат / замужем,0,82047,автомобиль,низкий


In [38]:
display(data_new['income_group'].value_counts())   # выделили 4 категории
display(data_new['income_group'].value_counts(normalize=True)*100)   

средний         11925
низкий           4463
высокий          3584
сверхвысокий     1482
Name: income_group, dtype: int64

средний         55.584040
низкий          20.802648
высокий         16.705509
сверхвысокий     6.907803
Name: income_group, dtype: float64

In [39]:
data_income = data_new.pivot_table(index='income_group', columns='debt', values='total_income', aggfunc='count').fillna(0) 
# сделали сводную таблицу
display(data_new.tail())
display(data_income.head(10)) 

,children,family_status,debt,total_income,purpose_new,income_group
21520,1,гражданский брак,0,224791,недвижимость,высокий
21521,0,женат / замужем,0,155999,автомобиль,средний
21522,1,гражданский брак,1,89672,недвижимость,низкий
21523,3,женат / замужем,1,244093,автомобиль,высокий
21524,2,женат / замужем,0,82047,автомобиль,низкий


debt,0,1
income_group,,
высокий,3332,252
низкий,4109,354
сверхвысокий,1376,106
средний,10896,1029


In [40]:
data_income[''] = data_income[1] / (data_income[0] + data_income[1])   # нашли долю должников у 4х категорий доходов
display(data_income.head(10)) 

display(data_new.groupby('income_group').agg({'debt': 'mean'}))  # проверка

debt,0,1,
income_group,,,
высокий,3332,252,0.070312
низкий,4109,354,0.079319
сверхвысокий,1376,106,0.071525
средний,10896,1029,0.086289


,debt
income_group,
высокий,0.070312
низкий,0.079319
сверхвысокий,0.071525
средний,0.086289


### Вывод

Как и логично было предположить, с повышением уровня дохода уровень надежности заемщика повышается. <br>
Заёмщики с низким и средним доходом допускают просрочку примерно одинаково.

In [41]:
pd.options.display.float_format = "{:.5f}".format
data_new['total_income'].describe()

count     21454.00000
mean     165225.60063
std       98021.02273
min       20667.00000
25%      107623.00000
50%      145017.00000
75%      195813.25000
max     2265604.00000
Name: total_income, dtype: float64

<hr style="border: 2px solid green;" />

### - Как разные цели кредита влияют на его возврат в срок?

In [42]:
data_new = data[['children','family_status', 'debt','total_income','purpose_new']]   # создаем новую рабочую таблицу
 
data_purpose = data_new.pivot_table(index='purpose_new', columns='debt', values='total_income', aggfunc='count').fillna(0)
display(data_purpose.head())



debt,0,1
purpose_new,,
автомобиль,3903,403
недвижимость,10029,782
образование,3643,370
свадьба,2138,186


In [43]:
data_purpose[''] = data_purpose[1] / (data_purpose[0] + data_purpose[1])   # нашли д 
display(data_purpose.head()) 
 

debt,0,1,
purpose_new,,,
автомобиль,3903,403,0.09359
недвижимость,10029,782,0.07233
образование,3643,370,0.09220
свадьба,2138,186,0.08003


In [44]:
display(data_new.groupby('purpose_new').agg({'debt': 'mean'}))  # проверка

,debt
purpose_new,
автомобиль,0.09359
недвижимость,0.07233
образование,0.09220
свадьба,0.08003


### Вывод

Кредиты на недвижимость и на свадьбу имеют больший процент возврата, чем у кредитов на автомобили и образование. <br>
Но не стоит забывать, что заявители могут скрывать свою основную цель. <br>
И объем кредита на свадьбу обычно небольшой, поэтому его хорошо отдают.

### Шаг 4. Общий вывод

В ходе исследования надёжности заёмщиков изучили статистику платёжеспособности клиентов банка и обработали её. Убрали дубликаты, пропуски, привели данные к нужному типу. Столбец с целями кредитов лемматизировали и выделили четыре основных.
Исходя из заданного вопроса категоризировали данные по наличию детей и доходам.

Все перечисленные факторы имеют связь с возвратом кредита: <br>
Наличие детей может быть причиной задержки оплаты кредита. <br>
Люди, состоящие в браке, более дисциплированны и меньше допускают просрочек. <br>
Уровень дохода прямо пропорционален уровню надежности заемщика.  <br> 
Кредиты на автомобили и образование возвращают хуже, чем на недвижимость и свадьбы. <br>

Ненадёжный заёмщик - это заёмщик, берущий деньги на недвижимость, не состоящий в официальном браке, имеющий одного или двух детей и низкий доход. 





